In [11]:
!pip install pyautogen  sentence_transformers markdownify pypdf -q --progress-bar off

In [13]:
!pip install chromadb==0.5.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 43.2 MB/s eta 0:00:00
  Attempting uninstall: chroma-hnswlib
    Found existing installation: chroma-hnswlib 0.7.6
    Uninstalling chroma-hnswlib-0.7.6:
      Successfully uninstalled chroma-hnswlib-0.7.6
  Attempting uninstall: chromadb
    Found existing installation: chromadb 0.5.15
    Uninstalling chromadb-0.5.15:
      Successfully uninstalled chromadb-0.5.15


In [1]:
#Importation des librairies
import chromadb # Used to create a vector database for storing and searching embeddings
from google.colab import userdata # Allows access to user information in Google Colab
from autogen import Cache # Used to manage the caching of results to optimize performance


from autogen import ConversableAgent # Base class for creating a chat agent
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent # Agent assistant to retrieve information
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent # Agent representing the user, retrieves data

# Language model configuration (LLM)
llm_config={
    "model": "gpt-3.5-turbo",
    "api_key": userdata.get('OPENAI_API_KEY'),
    "temperature": 0.7
}

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:
retriever_config = {
    "task": "qa", # Definition of the agent's task: here, “qa” for questions and answers
    "model": llm_config["model"], # Use of a model configuration previously defined under “model”
    "client": chromadb.PersistentClient(path="/content/tmp/db"),  # Configure chromadb client to store and search for embeddings
    "embedding_model": "all-mpnet-base-v2", # Pre-trained embedding model for encoding phrases into vectors
    # List of paths (URLs) to text documents that will be used for questions and answers
    "docs_path": [
        "https://github.com/Azure-Samples/azure-search-sample-data/blob/main/nasa-e-book/earth-txt-10/page-11.txt",
        "https://github.com/Azure-Samples/azure-search-sample-data/blob/main/nasa-e-book/earth-txt-10/page-13.txt",
        "https://github.com/Azure-Samples/azure-search-sample-data/blob/main/nasa-e-book/earth-txt-10/page-15.txt",
        "https://github.com/Azure-Samples/azure-search-sample-data/blob/main/nasa-e-book/earth-txt-10/page-17.txt",
        "https://github.com/Azure-Samples/azure-search-sample-data/blob/main/nasa-e-book/earth-txt-10/page-19.txt",
        "https://github.com/Azure-Samples/azure-search-sample-data/blob/main/nasa-e-book/earth-txt-10/page-21.txt",
        "https://github.com/Azure-Samples/azure-search-sample-data/blob/main/nasa-e-book/earth-txt-10/page-23.txt"
    ],
}

rag_proxy_agent = RetrieveUserProxyAgent(
    name="rag_proxy_agent", # Agent's name
    human_input_mode="NEVER", # The agent operates autonomously
    retrieve_config=retriever_config, # Use the configuration defined above for information retrieval
    code_execution_config=False,  # Set to False if you don't want to execute any code
)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
rag_assistant = RetrieveAssistantAgent(
    name="rag_assistant", # Agent's name
    system_message="You are a helpful AI assistant.", # System message for the agent
    llm_config={
        "timeout":600, # (expiry time) in seconds: the agent has 600 seconds to respond to a request before it expires
        "config_list":[llm_config]
    },
)

<ipython-input-3-8a56fa4ebc74>:1: DeprecationWarning: The RetrieveAssistantAgent is deprecated. Please use the AssistantAgent instead.
  rag_assistant = RetrieveAssistantAgent(


In [11]:


# Define the query or question to ask the assistant
query= "What is the impact of volcanic eruptions on cloud formation?"



# Use disk-based caching to store results or data on the disk to prevent re-running the same operations multiple times if the same data is requested later
with Cache.disk() as cache:


  rag_proxy_agent.initiate_chat(
      rag_assistant,# The assistant agent that will respond to the query
      problem=query,# The query or question to ask the assistant
      n_results=2,# Set the number of results or responses to retrieve from the assistant (2 in this case)
      message=rag_proxy_agent.message_generator,# Use the proxy agent's message generator to dynamically generate the message to send
      max_turns=5,# Limit the conversation to a maximum of 5 exchanges (turns) between the agents

  )

VectorDB returns doc_ids:  [['168df6a8', '3c43bc93']]
Adding content of doc 168df6a8 to context.
Adding content of doc 3c43bc93 to context.
rag_proxy_agent (to rag_assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
You must give as short an answer as possible.

User's question is: What is the impact of volcanic eruptions on cloud formation?

Context is: * [Code](/Azure-Samples/azure-search-sample-data)
* [Issues](/Azure-Samples/azure-search-sample-data/issues)
* [Pull requests](/Azure-Samples/azure-search-sample-data/pulls)
* [Actions](/Azure-Samples/azure-search-sample-data/actions)
* [Projects](/Azure-Samples/azure-search-sample-data/projects)
* [Security](/Azure-Samples/azure-search-sample-data/security)
* [Insights](/Azure-Samples/azure-search-sample-data/pulse)

 

   Files
----